<a href="https://colab.research.google.com/github/alvaroleva/TFG_collabs/blob/main/p01_LoadMergeSumm_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TFG |AI-based classification of symptomatic subjects with Brugada Syndrome from ECG-derived Markers and clinical data**
## June 2023, Álvaro Leva Ligero. 

> Director/s: Elena Arbelo, Flavio Palmieri, Pedro Gomis
--------

## 0. Import biomarker data

In [ ]:
import pandas as pd 
import numpy as np
#Code needed to import the Data from GoogleDrive
from google.colab import files
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

data_bmk_url = 'https://docs.google.com/spreadsheets/d/1o4RjeTf49jVR5YKW0LgxBobenazR3cgp/edit?usp=share_link&ouid=111241553451068058703&rtpof=true&sd=true'
id = data_bmk_url.split("/")[-2] #get ID
downloaded = drive.CreateFile({'id':id}) #download from ID
downloaded.GetContentFile('Biomarkers_12leads.xlsx') #Get the file in the Colab working directory

data_clc_url = 'https://docs.google.com/spreadsheets/d/1fM0X1WZkGNA2jVgFaeEHWuzUAfgTikZj/edit?usp=share_link&ouid=111241553451068058703&rtpof=true&sd=true'
id = data_clc_url.split("/")[-2] #get ID
downloaded = drive.CreateFile({'id':id}) #download from ID
downloaded.GetContentFile('2023.05.04_BrS_AI4BrS.xlsx') #Get the file in the Colab working directory

## 1. Load Raw Data

### 1.1 Biomarker Data

In [ ]:
data_bmk = pd.read_excel('Biomarkers_12leads.xlsx') #load biomarker data
data_bmk

,BH,Pavg_I,AreaQRSabs_I,qt_I,QTc_I,ST_0_I,ST_60_I,ST_slope_I,PR_I,LP_QRSd_I,...,AreaQRSabs_V6,qt_V6,QTc_V6,ST_0_V6,ST_60_V6,ST_slope_V6,PR_V6,LP_QRSd_V6,LP_RMS40_V6,LP_LAS_V6
0,1,0.031307,30.223308,0.441,582.943186,-0.246917,-0.334133,-1.453612,NaN,194.0,...,16.879058,0.401,511.533029,0.095633,0.032433,-1.053320,0.200,79.0,15.154072,33.0
1,1,0.028199,31.307816,NaN,NaN,0.229631,0.323677,1.567421,0.266,235.0,...,16.931552,0.374,477.977227,0.064844,0.018609,-0.770583,NaN,80.0,14.579114,35.0
2,1,0.022241,12.202246,0.370,476.165151,0.004483,0.021451,0.282791,0.172,104.0,...,17.283096,0.402,462.726628,0.038386,-0.004283,-0.711148,0.199,78.0,23.108229,47.0
3,1,0.027147,11.260574,0.368,484.534239,0.005239,0.015902,0.177720,0.188,87.0,...,17.617875,0.429,490.494209,0.050398,0.002574,-0.797062,0.191,80.0,21.490465,32.0
4,1,0.011905,9.902881,0.376,477.711804,0.041665,0.073924,0.537640,0.171,112.0,...,19.010422,0.404,481.940820,0.061270,0.008722,-0.875803,0.190,78.0,15.928766,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31004,111,0.338212,25.725335,0.334,411.915115,-0.127636,-0.010983,1.944220,0.173,58.0,...,23.453971,0.311,379.959764,0.165617,0.071700,-1.565284,0.161,88.0,36.849871,15.0
31005,111,0.403447,27.993290,0.339,403.268611,-0.110133,0.001015,1.852461,0.173,58.0,...,23.254568,0.318,380.525699,0.180949,0.077530,-1.723652,0.162,92.0,26.983092,18.0
31006,111,0.430240,28.990620,0.317,366.664495,-0.061196,0.035431,1.610448,0.179,59.0,...,23.010498,0.322,364.570754,0.239293,0.125222,-1.901183,0.166,92.0,32.390284,19.0
31007,111,0.313369,24.488286,0.323,392.201684,-0.061657,0.033886,1.592379,0.177,58.0,...,21.939452,0.313,364.510772,0.284862,0.122045,-2.713622,0.167,90.0,32.048746,36.0


In [ ]:
print("Original number of patients included in the biomarker's dataset: {}".format( len( data_bmk.BH.unique() ) ))

Original number of patients included in the biomarker's dataset: 76


### 1.2 Clinical Data

From all the available clinical data provided by the original SPSS database from Hospital Clínic, the **most clinically important variables according to the criteria of  Dra. Elena Arbelo** will be used. 

In [ ]:
data_clc = pd.read_excel('2023.05.04_BrS_AI4BrS.xlsx') #load clinical data
data_clc = data_clc.drop('BH', axis=1) #drop unwanted column (not properly filled)
data_clc = data_clc.rename(columns={'COD_ANVERSO': 'BH'}) #Rename 'COD_ANVERSO' = 'BH'

#Define the independent clinical vars
clinical_vars = ['Diagnosis_age', 'Gender','Proband', 'FH_SCD', 'FH_SCD_bef_55', 'Symptoms_bef_Dg_DIC_wo_vagal','SD_VF_bef_Dg', 'sponECGtype1_all', 'AF_bef_Dg', 'EPS_inducib_dic' ]

#Finally define our clinical data matrix with the desired columns
data_clc = data_clc[ ['BH'] + list(clinical_vars)]
data_clc

,BH,Diagnosis_age,Gender,Proband,FH_SCD,FH_SCD_bef_55,Symptoms_bef_Dg_DIC_wo_vagal,SD_VF_bef_Dg,sponECGtype1_all,AF_bef_Dg,EPS_inducib_dic
0,BH0116,53.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BH0115,24.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,BH0113,44.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,BH0111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BH0110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
87,BH0005,52.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
88,BH0004,30.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
89,BH0003,36.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
90,BH0002,51.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Merge biomarker and clinical datasets

a. Redefine data format and data type.

In [ ]:
# Redefine BH name to be the same in both datasets <number>
data_clc.BH= data_clc.BH.str.replace('BH', '').str.lstrip('0').astype(int)

In [ ]:
#See how many patients contain each variable category 
print('From a total of {} patients \n------------------------------'.format(len(data_clc)))
for var in data_clc.columns[2:]:
  print(data_clc[var].value_counts())

#Create a clinical dataset with categorical data
data_clc_cat = data_clc.copy()
dic_vars = ['Gender', 'Proband', 'FH_SCD', 'FH_SCD_bef_55', 'Symptoms_bef_Dg_DIC_wo_vagal', 'SD_VF_bef_Dg', 'EPS_inducib_dic', 'sponECGtype1_all', 'AF_bef_Dg' ] #dicotomic variables
for var in dic_vars:
  data_clc_cat[var] = pd.Categorical(data_clc_cat[var], categories = [0, 1], ordered = False)

#Set a unique format for missing values
data_clc = data_clc.replace(-99, np.nan)
data_clc_cat = data_clc_cat.replace(-99, np.nan)

print('\n----------------------------\n', data_clc_cat.dtypes)
data_clc_cat

From a total of 92 patients 
------------------------------
0.0    60
1.0    26
Name: Gender, dtype: int64
1.0    66
0.0    19
Name: Proband, dtype: int64
0.0    57
1.0    28
Name: FH_SCD, dtype: int64
0.0    59
1.0    24
Name: FH_SCD_bef_55, dtype: int64
0.0    72
1.0    12
Name: Symptoms_bef_Dg_DIC_wo_vagal, dtype: int64
0.0    80
1.0     3
Name: SD_VF_bef_Dg, dtype: int64
0.0    50
1.0    35
Name: sponECGtype1_all, dtype: int64
0.0    74
1.0     8
Name: AF_bef_Dg, dtype: int64
 0.0     56
-99.0    16
 1.0     10
Name: EPS_inducib_dic, dtype: int64

----------------------------
 BH                                 int64
Diagnosis_age                    float64
Gender                          category
Proband                         category
FH_SCD                          category
FH_SCD_bef_55                   category
Symptoms_bef_Dg_DIC_wo_vagal    category
SD_VF_bef_Dg                    category
sponECGtype1_all                category
AF_bef_Dg                       category
EP

,BH,Diagnosis_age,Gender,Proband,FH_SCD,FH_SCD_bef_55,Symptoms_bef_Dg_DIC_wo_vagal,SD_VF_bef_Dg,sponECGtype1_all,AF_bef_Dg,EPS_inducib_dic
0,116,53.0,1,1,0,0,0,0,0,0,0
1,115,24.0,1,0,1,1,0,0,0,0,0
2,113,44.0,0,1,1,0,0,0,1,0,0
3,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110,38.0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
87,5,52.0,0,1,0,0,0,0,1,0,0
88,4,30.0,0,1,0,0,0,0,1,0,0
89,3,36.0,0,1,0,0,1,0,0,0,0
90,2,51.0,1,1,0,0,0,0,0,0,0


b. Merge both biomarker and clinical datasets (only the coexisting BH patients remain)

In [ ]:
data = pd.merge(data_clc,data_bmk, on='BH', how='inner') 

#Drop the patient's from which only clinical data / biomarker data exist
data = data.sort_values(by='BH') #sort values
data = data.reset_index(drop=True)

#Check with patients lack of clinical data
print(data.loc[data['Gender'].isnull(), 'BH'].unique())

#remove for now this patients
data = data[(data.BH != 86) & (data.BH != 111) ]


#Create a data with categorical values for the summary
data_cat = pd.merge(data_clc_cat,data_bmk, on='BH', how='inner') 

#Drop the patient's from which only clinical data / biomarker data exist
data_cat = data_cat[(data_cat.BH != 86) & (data_cat.BH != 111)]
data_cat = data_cat.sort_values(by='BH') #sort values
data_cat = data_cat.reset_index(drop=True)

# #Save the merged dataset in excel
# from google.colab import files

# data.to_excel('dsFinal_biomarkersPlusClinical.xlsx')

# # download the file to your local PC
# files.download('dsFinal_biomarkersPlusClinical.xlsx')

#Show data
data

[ 86 111]


,BH,Diagnosis_age,Gender,Proband,FH_SCD,FH_SCD_bef_55,Symptoms_bef_Dg_DIC_wo_vagal,SD_VF_bef_Dg,sponECGtype1_all,AF_bef_Dg,...,AreaQRSabs_V6,qt_V6,QTc_V6,ST_0_V6,ST_60_V6,ST_slope_V6,PR_V6,LP_QRSd_V6,LP_RMS40_V6,LP_LAS_V6
0,1,49.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,19.404479,0.426,479.634120,0.114049,0.099740,-0.238484,0.216,79.0,15.969734,31.0
1,1,49.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,19.724454,0.426,508.969256,0.131909,0.098811,-0.551632,0.219,82.0,17.210309,33.0
2,1,49.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,19.236563,0.407,493.999389,0.135950,0.077048,-0.981696,0.215,82.0,16.221674,34.0
3,1,49.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,20.491644,0.412,470.283612,0.123019,0.058792,-1.070458,0.213,81.0,16.741394,33.0
4,1,49.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,20.532568,0.419,478.138960,0.118188,0.069585,-0.810037,0.214,81.0,16.993143,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26678,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,41.421747,0.396,394.782956,0.014946,0.071510,0.942732,0.199,111.0,10.362226,44.0
26679,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,40.890811,0.397,390.143153,0.018117,0.071957,0.897337,0.196,112.0,10.092416,44.0
26680,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,41.107990,0.396,386.601336,0.018687,0.076640,0.965896,0.196,111.0,10.612109,43.0
26681,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,41.602687,0.397,392.389449,0.023788,0.078426,0.910627,0.196,112.0,10.042074,43.0


#2.Data summary

a. Statistical summary for numerical data. 

In [ ]:
print('Total number of patients of the final dataset: {}'.format( len( data_cat.BH.unique() ) ))

#See an statistical summary of our overall numerical data
summary = data_cat.describe().round(3)
summary.loc['NaN(%)'] = round(data_cat.isna().sum() / data_cat.shape[0]*100,2)
summary

Total number of patients of the final dataset: 64


,BH,Diagnosis_age,Pavg_I,AreaQRSabs_I,qt_I,QTc_I,ST_0_I,ST_60_I,ST_slope_I,PR_I,...,AreaQRSabs_V6,qt_V6,QTc_V6,ST_0_V6,ST_60_V6,ST_slope_V6,PR_V6,LP_QRSd_V6,LP_RMS40_V6,LP_LAS_V6
count,26304.000,26304.000,26130.000,26130.000,25367.000,25367.000,26130.000,26130.000,26130.000,23836.000,...,26099.000,25584.000,25584.000,26099.000,26099.000,26099.000,23381.000,26099.000,26099.000,26099.000
mean,48.940,43.209,0.087,18.753,0.382,438.646,0.011,0.030,0.309,0.185,...,37.724,0.383,425.935,0.108,0.116,0.127,0.191,105.457,25.658,38.648
std,37.032,12.939,0.085,12.080,0.045,61.007,0.087,0.079,1.024,0.040,...,30.368,0.048,56.900,0.158,0.124,2.036,0.039,28.537,38.818,12.910
min,1.000,10.000,0.000,1.415,0.199,205.434,-3.256,-3.482,-16.872,0.112,...,1.399,0.187,193.634,-3.238,-1.287,-67.125,0.104,39.000,0.354,0.000
25%,18.000,35.000,0.029,11.004,0.353,400.353,-0.013,0.007,-0.168,0.159,...,22.886,0.350,391.150,0.039,0.052,-0.945,0.169,90.000,8.438,31.000
50%,35.000,43.000,0.063,15.900,0.374,424.439,0.017,0.029,0.325,0.177,...,32.690,0.379,413.005,0.091,0.094,0.098,0.184,102.000,16.281,41.000
75%,94.000,51.000,0.113,23.673,0.400,459.388,0.046,0.049,0.807,0.199,...,45.525,0.407,448.007,0.157,0.161,1.157,0.205,114.000,27.673,51.000
max,110.000,76.000,3.972,356.537,0.691,889.388,2.975,2.705,15.783,0.340,...,2568.658,0.654,892.331,9.072,5.045,33.868,0.340,405.000,827.003,51.000
NaN(%),0.000,0.000,0.660,0.660,3.560,3.560,0.660,0.660,0.660,9.380,...,0.780,2.740,2.740,0.780,0.780,0.780,11.110,0.780,0.780,0.780


**b. Python code to save a summary of the biomarkers lead by lead in different Excel sheets:**

In [ ]:
import pandas as pd
from openpyxl import Workbook
from google.colab import files

# generate an excel workbook
workbook = Workbook()
writer = pd.ExcelWriter('Biomarker_lead_summary.xlsx', engine='openpyxl')
writer.book = workbook

leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'] #lead labels
# iterate through the columns in groups of 11 biomarkers (11 bmk / lead)
for i in range(1, len(data_bmk.columns)-11, 11):
  
    cols = data_bmk.columns[i:i+11]

    # compute the summary statistics for the selected columns
    summary = data_bmk[cols].describe().round(3)
    summary.loc['NaN(%)'] = round(data_bmk[cols].isna().sum() / data_bmk.shape[0] * 100, 2)

    # get the lead label for the current group
    lead_label = leads[i // 11]

    sheet_name = f'lead_{lead_label}' #sheet name as "lead_<lead label>

    summary.to_excel(writer, sheet_name=sheet_name)

# Save the workbook
writer.save()

# Download the Excel file
files.download('Biomarker_lead_summary.xlsx')


<ipython-input-10-9cc383f64859>:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = workbook
<ipython-input-10-9cc383f64859>:28: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#See the first 10 variables with more NaNs in percentatge (%)
pd.DataFrame( round(summary.loc['NaN(%)',:].nlargest(10),2) )

,NaN(%)
PR_V5,10.42
qt_V5,3.15
QTc_V5,3.15
Pavg_V5,0.76
AreaQRSabs_V5,0.76
ST_0_V5,0.76
ST_60_V5,0.76
ST_slope_V5,0.76
LP_QRSd_V5,0.76
LP_RMS40_V5,0.76


b. Categorical summary for categorical data. 

In [ ]:
#See a summary of categorical data
cat_summ = data_cat.describe(include=['category'])
cat_summ.loc['freq', :] = cat_summ.loc['freq', :] / cat_summ.loc['count', :] * 100
cat_summ = cat_summ.rename(index={'freq': 'freq top (%)'})
cat_summ.astype(int)

,Gender,Proband,FH_SCD,FH_SCD_bef_55,Symptoms_bef_Dg_DIC_wo_vagal,SD_VF_bef_Dg,sponECGtype1_all,AF_bef_Dg,EPS_inducib_dic
count,26304,26304,26304,25832,26304,26304,26304,25832,21417
unique,2,2,2,2,2,2,2,2,2
top,0,1,0,0,0,0,0,0,0
freq top (%),72,80,69,72,84,96,56,91,82


With the categorical summary the main limitation of our dataset is firstly seen.
> **Symptomatic patients are scarce.**


> Therefore, the target variable will be considered ``Symptoms_bef_Dg_DIC_wo_vagal``, to determine the presence (1) or abscence (0) of symptoms. 


In [ ]:
import plotly.express as px
#See data unbalancing
target_var_counts = data.Symptoms_bef_Dg_DIC_wo_vagal.value_counts()
print(target_var_counts, sep = '\n')
print('\nA clear unbalancing is observed between the classes: \n-Symptomatic (1): {:.2f}%\n-Asymptomatic (0): {:.2f}%'.format(
    target_var_counts[0] / sum(target_var_counts)*100, 100 - target_var_counts[0] / sum(target_var_counts)*100
))

fig = px.pie(data, values= data.Symptoms_bef_Dg_DIC_wo_vagal.value_counts(), names = ['Asymptomatic', 'Symptomatic'], width = 400, height = 400)
fig.show()

0.0    22215
1.0     4089
Name: Symptoms_bef_Dg_DIC_wo_vagal, dtype: int64

A clear unbalancing is observed between the classes: 
-Symptomatic (1): 84.45%
-Asymptomatic (0): 15.55%


In [ ]:
#Rename 'Symptoms_bef_Dg_DIC_wo_vagal' as 'symp', our target variable
data = data.rename(columns={'Symptoms_bef_Dg_DIC_wo_vagal': 'symp'})

#Move it to the last column for better readability
symp_col = data.pop('symp')
data.insert( len(data.columns), 'symp', symp_col)

#Do the same for data_cat
data_cat = data_cat.rename(columns={'Symptoms_bef_Dg_DIC_wo_vagal': 'symp'})
symp_col = data_cat.pop('symp')
data_cat.insert( len(data_cat.columns), 'symp', symp_col)


#Show data results
data

<ipython-input-18-f89f763d590d>:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



,BH,Diagnosis_age,Gender,Proband,FH_SCD,FH_SCD_bef_55,SD_VF_bef_Dg,sponECGtype1_all,AF_bef_Dg,EPS_inducib_dic,...,qt_V6,QTc_V6,ST_0_V6,ST_60_V6,ST_slope_V6,PR_V6,LP_QRSd_V6,LP_RMS40_V6,LP_LAS_V6,symp
0,1,49.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.426,479.634120,0.114049,0.099740,-0.238484,0.216,79.0,15.969734,31.0,0.0
1,1,49.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.426,508.969256,0.131909,0.098811,-0.551632,0.219,82.0,17.210309,33.0,0.0
2,1,49.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.407,493.999389,0.135950,0.077048,-0.981696,0.215,82.0,16.221674,34.0,0.0
3,1,49.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.412,470.283612,0.123019,0.058792,-1.070458,0.213,81.0,16.741394,33.0,0.0
4,1,49.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.419,478.138960,0.118188,0.069585,-0.810037,0.214,81.0,16.993143,32.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26678,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.396,394.782956,0.014946,0.071510,0.942732,0.199,111.0,10.362226,44.0,0.0
26679,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.397,390.143153,0.018117,0.071957,0.897337,0.196,112.0,10.092416,44.0,0.0
26680,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.396,386.601336,0.018687,0.076640,0.965896,0.196,111.0,10.612109,43.0,0.0
26681,110,38.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.397,392.389449,0.023788,0.078426,0.910627,0.196,112.0,10.042074,43.0,0.0


> ### [Next script](https://colab.research.google.com/drive/1ebdLWv-O7FurtHxEVpgi4ute8-b-gOSn#scrollTo=uNzXjma_temh) : P02 | Feature Selection and Data Visualization.